In [1]:
%load_ext autoreload
%autoreload 2
from data_loader import load_datasets, create_data_loaders
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import matplotlib.pyplot as plt
from training_pipeline import repeat_training, train_with_different_parameters
from serialization import load
from cnn_model import Net
import numpy as np
import random

SEED = 42


In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [3]:
set_seed(SEED)
batch_size = 64

In [4]:
train, val, test = load_datasets((32, 32))
train_dat, val_dat, test_dat = create_data_loaders(train, val, test, batch_size)

### Learning rate test

In [5]:
set_seed(SEED)
train_with_different_parameters(5, Net, 10, train_dat, val_dat, test_dat, 'cuda', batch_size, lrs=[i/500 for i in range(1,6)])

training iteration: 1 of 5
starting training...
epoch: 1, training loss: 0.027251175389024947, training accuracy: 34.022222222222226
epoch: 1, validation loss: 1.509980493121677e-05, validation accuracy: 43.827777777777776
model saved

epoch: 2, training loss: 0.024445606304539576, training accuracy: 41.99888888888889
epoch: 2, validation loss: 9.555004702674018e-06, validation accuracy: 46.306666666666665
model saved

epoch: 3, training loss: 0.023423221050368415, training accuracy: 44.88666666666666
epoch: 3, validation loss: 1.291909482744005e-05, validation accuracy: 48.26444444444444

epoch: 4, training loss: 0.02288557147582372, training accuracy: 46.074444444444445
epoch: 4, validation loss: 1.3570190800560846e-05, validation accuracy: 51.202222222222225

epoch: 5, training loss: 0.022509137505955165, training accuracy: 47.492222222222225
epoch: 5, validation loss: 9.595686859554715e-06, validation accuracy: 50.952222222222225

epoch: 6, training loss: 0.022221468182404835, trai

### Dropout test

In [ ]:
set_seed(SEED)
train_with_different_parameters(5, Net, 10, train_dat, val_dat, test_dat, 'cuda', batch_size, dropouts=[i/10 for i in range(3,8)])

### Betas tests

In [ ]:
set_seed(SEED)
train_with_different_parameters(5, Net, 10, train_dat, val_dat, test_dat, 'cuda', batch_size, betas=[(1-i/10, 1-i/1000) for i in range(2,6)])